<div style="display: flex; align-items: center; justify-content: center; flex-wrap: wrap;">
    <div style="flex: 1; max-width: 400px; display: flex; justify-content: center;">
        <img src="https://i.ibb.co/JBPWVYR/Logo-Nova-IMS-Black.png" style="max-width: 50%; height: auto; margin-top: 50px; margin-bottom: 50px;margin-left: 6rem;">
    </div>
    <div style="flex: 2; text-align: center; margin-top: 20px;margin-left: 8rem;">
        <div style="font-size: 28px; font-weight: bold; line-height: 1.2;">
            <span style="color: #22c1c3;">DL Project |</span> <span style="color: #08529C;">Predicting Rare Species from Images using Deep Learning</span>
        </div>
        <div style="font-size: 17px; font-weight: bold; margin-top: 10px;">
            Spring Semester | 2024 - 2025
        </div>
        <div style="font-size: 17px; font-weight: bold;">
            Master in Data Science and Advanced Analytics
        </div>
        <div style="margin-top: 20px;">
            <div>André Silvestre, 20240502</div>
            <div>Diogo Duarte, 20240525</div>
            <div>Filipa Pereira, 20240509</div>
            <div>Maria Cruz, 20230760</div>
            <div>Umeima Mahomed, 20240543</div>
        </div>
        <div style="margin-top: 20px; font-weight: bold;">
            Group 37
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 1px; color: white; border-radius: 500px; text-align: center;">
</div>

## **📚 Libraries Import**

In [3]:
# System imports
import os
import sys
import time
import datetime
from tqdm import tqdm
from typing_extensions import Self, Any      # For Python 3.10
# from typing import Self, Any               # For Python >3.11

from pathlib import Path

# Data manipulation imports
import numpy as np
import pandas as pd  
import warnings
warnings.filterwarnings("ignore")

# Data visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning imports
import tensorflow as tf
from keras.ops import add
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Image processing imports (Data Augmentation)
from tensorflow.keras.layers import Rescaling, RandAugment

# Pretrained model imports
from tensorflow.keras.applications import ConvNeXtBase

# Evaluation imports
from sklearn.metrics import confusion_matrix, classification_report
from keras.metrics import CategoricalAccuracy, AUC, F1Score, Precision, Recall

# Other imports
from itertools import product

# Image processing imports
from matplotlib.image import imread
from PIL import Image

# Set the style of the visualization
pd.set_option('future.no_silent_downcasting', True)   # use int instead of float in DataFrame
pd.set_option("display.max_columns", None)            # display all columns

# Disable warnings (FutureWarning)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# For better resolution plots
%config InlineBackend.figure_format = 'retina'
# Setting seaborn style
sns.set_theme(style="white")

# Set random seed for reproducibility
np.random.seed(2025)

2025-03-29 22:17:53.800039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743286674.160297   43886 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743286674.230800   43886 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-29 22:17:54.635123: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
print("TensorFlow Version:", tf.__version__)
print("Is TensorFlow built with CUDA?", tf.test.is_built_with_cuda())
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("GPU Device Name:", tf.test.gpu_device_name())

if tf.test.is_built_with_cuda():
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

TensorFlow Version: 2.18.0
Is TensorFlow built with CUDA? True
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU Device Name: /device:GPU:0


I0000 00:00:1743286678.889168   43886 gpu_device.cc:2022] Created device /device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
# Auxiliary function to display multiple dataframes side by side
# Source: https://python.plainenglish.io/displaying-multiple-dataframes-side-by-side-in-jupyter-lab-notebook-9a4649a4940
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args, super_title, titles=cycle([''])):
    """
    :param args: Variable number of DataFrame objects to be displayed side by side.
    :param super_title: The main title to be displayed at the top of the combined view.
    :param titles: An iterable containing titles for each DataFrame to be displayed. Defaults to an infinite cycle of empty strings.
    
    :return: None. The function generates and displays HTML content side by side for given DataFrames.
    """
    html_str = ''
    html_str += f'<h1 style="text-align: left; margin-bottom: -15px;">{super_title}</h1><br>'
    html_str += '<div style="display: flex;">'
    for df, title in zip(args, chain(titles, cycle(['</br>']))):
        html_str += f'<div style="margin-right: 20px;"><h3 style="text-align: center;color:#555555;">{title}</h3>'
        html_str += df.to_html().replace('table', 'table style="display:inline; margin-right: 20px;"')
        html_str += '</div>'
    html_str += '</div>'
    display_html(html_str, raw=True)

## **🧮 Import Databases**

In [ ]:
# # Run in Google Collab to download the dataset already splitted
# # Source: https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drivez
# # Download the file from Google Drive using wget
# !wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
#   "https://drive.usercontent.google.com/download?id=1EYwK5a0wzwg5dHSFhn38zRsi7cihEXtc&export=download" -O- | \
#   sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p' > /tmp/confirm.txt

# # Read the confirmation token from the temporary file
# with open('/tmp/confirm.txt', 'r') as f:
#     confirm_token = f.read().strip()

# # Download the file using the confirmation token and cookies
# !wget --load-cookies /tmp/cookies.txt \
#   "https://drive.usercontent.google.com/download?id=1EYwK5a0wzwg5dHSFhn38zRsi7cihEXtc&export=download&confirm={confirm_token}" \
#   -O /content/RareSpecies_Split.zip

# # Clean up temporary files
# !rm /tmp/cookies.txt /tmp/confirm.txt

# # List files in the /content directory to verify the download
# !ls -lh /content/

# # Unzip the downloaded file
# !unzip /content/RareSpecies_Split.zip -d /content/

# # List the unzipped files to verify
# !ls -lh /content/

In [7]:
# Define the path to the data
train_dir = Path("data/train")
val_dir = Path("data/val")
test_dir = Path("data/test")

# For Google Collab
# train_dir = Path("/content/RareSpecies_Split/train")
# val_dir = Path("/content/RareSpecies_Split/val")
# test_dir = Path("/content/RareSpecies_Split/test")

In [8]:
# Image Generators
n_classes = 202                                     # Number of classes (we already know this based on previous notebook)
image_size = (224, 224)                             # Size of the images
img_width, img_height = 224, 224               
batch_size = 16                                     # Batch size
input_shape = (img_width, img_height, 3)            # Input shape of the model
value_range = (0.0, 1.0)                            # Range of pixel values

# Data generators with built-in rescaling (no augmentation yet)

# Training data generator
train_datagen = image_dataset_from_directory(
    train_dir,                                      # Path to the directory
    labels='inferred',                              # Type of labels to generate (inferred = from the directory structure)
    label_mode='categorical',                       # Type of labels to generate (categorical = 'float32' tensor of shape (batch_size, num_classes), representing a one-hot encoding of the class index.)
    color_mode='rgb',                               # Color mode to read images
    batch_size=batch_size,                          # Size of the batches of data
    image_size=(img_width, img_height),             # Size of the images to read
    shuffle=True,                                   # Whether to shuffle the data
    seed=2025,                                      # Random seed for shuffling and transformations
    interpolation='bilinear',                       # Interpolation method to resample the image
)

# Validation data generator
val_datagen = image_dataset_from_directory(val_dir, labels='inferred', label_mode='categorical', color_mode='rgb', batch_size=batch_size,
                                           image_size=(img_width, img_height), shuffle=True, seed=2025, interpolation='bilinear')

# Test data generator
test_datagen = image_dataset_from_directory(test_dir, labels='inferred', label_mode='categorical', color_mode='rgb', batch_size=batch_size,
                                            image_size=(img_width, img_height), shuffle=True, seed=2025, interpolation='bilinear')

Found 8388 files belonging to 202 classes.


I0000 00:00:1743286684.745621   43886 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Found 1797 files belonging to 202 classes.
Found 1798 files belonging to 202 classes.


# <a class='anchor' id='3'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 140px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 32px; font-family: 'Avenir Next LT Pro', sans-serif;">
        <b>3 | Modeling - ConvNetXt</b></h1></center>
</div>

<br><br>

<center><img src="" style="width: 700px"></center>

# **💡 Modeling**

In [ ]:
# Baseline Model
class ConvNetXt(Model):
    """Convolutional Neural Network for image classification | ConvNetXt Pretrained Model."""
    def __init__(self: Self) -> None:
        """Initializes the model."""
        
        # Call the parent class constructor
        super().__init__()
        
        # Rescaling layer
        self.rescale_layer = Rescaling(scale= 1 / 255.0, name="Rescale_Layer")    # Rescales pixel values to [0, 1]
        
        # Augmentation layer
        self.augmentation = RandAugment(value_range=(0, 1), name="RandAugment_Layer")  # Applies random augmentations to the input images
        
        # Pre-trained architecture
        self.pre_trained_architecture  = ConvNeXtBase(weights="imagenet")
        
        # Classification head
        self.flatten = Flatten(name="Flatten_Layer")                                  # Flattens the output for the dense layer
        # self.dropout = Dropout(0.3)                                                   # Prevents overfitting
        self.dense = Dense(n_classes, activation='softmax', name="Dense_Layer")       # Outputs probabilities for 202 classes
    
    def call(self, inputs, training=False):
        """Defines the forward pass."""
        x = self.rescale_layer(inputs)                      # Rescale the input images
        x = self.augmentation(x)                            # Apply augmentations
        x = self.pre_trained_architecture(x)                # Pass through the pre-trained architecture
        x = self.flatten(x)                                # Flatten the output
        # x = self.dropout(x)                               # Apply dropout
        x = self.dense(x)                                  # Pass through the dense layer
        return x                                           # Return the output
        
        
# Instantiate the model
model = ConvNetXt()
inputs = Input(shape=(224, 224, 3))
_ = model.call(inputs)
model.summary()

In [ ]:
# Compile model
optimizer = SGD(learning_rate=0.01, name="Optimizer")                  # SGD with decay for stability
loss = CategoricalCrossentropy(name="Loss")                            # Suitable for multi-class one-hot labels
metrics = [CategoricalAccuracy(name="accuracy"), 
           Precision(name="precision"),
           Recall(name="recall"), 
           F1Score(average="macro", name="f1_score"),
           AUC(name="auc")]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks
callbacks = [
    ModelCheckpoint("checkpoint.keras", monitor="val_loss", save_best_only=True, verbose=0),        # Save best model
    CSVLogger("metrics.csv"),                                                                       # Log training metrics
    LearningRateScheduler(lambda epoch, lr: lr * 0.95)                                              # Exponential decay for learning rate
]

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_datagen, batch_size = batch_size, epochs=5, validation_data=val_datagen, callbacks=callbacks, verbose=2)
train_time = round(time.time() - start_time, 2)

print(f"Training completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

---

### <a class='anchor' id='3_1'></a> <a class='anchor' id='3_2'></a>  **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# Function to plot training and validation metrics
def plot_metrics(history):
    """Plots training and validation loss, accuracy, and F1 score."""
    fig, ax = plt.subplots(1, 3, figsize=(18, 4))
    metrics = [('loss', 'Loss'), ('accuracy', 'Accuracy'), ('f1_score', 'F1 Score')]
    for i, (metric, title) in enumerate(metrics):
        ax[i].plot(history.history[metric], label='Train', color='#22c1c3')
        ax[i].plot(history.history[f'val_{metric}'], label='Validation', color='#090979')
        ax[i].set_title(title, fontsize=14, fontweight='bold')
        ax[i].set_xlabel('Epoch', fontsize=12)
        ax[i].set_ylabel(title, fontsize=12)
        ax[i].legend()
        sns.despine(top=True, right=True)
    plt.tight_layout()
    plt.show()

print("\nTraining Metrics Plot:")
plot_metrics(history)

In [ ]:
# Evaluate on test set
train_results = model.evaluate(train_datagen, batch_size=batch_size, return_dict=True, verbose=2)
val_results = model.evaluate(val_datagen, batch_size=batch_size, return_dict=True, verbose=2)
test_results = model.evaluate(test_datagen, batch_size=batch_size, return_dict=True, verbose=2)

In [ ]:
# Collect results in DataFrame
results_df = pd.DataFrame({
    "Models": ["Baseline Model"],
    "Time of Execution": [train_time],
    "Training Set Accuracy": [train_results['accuracy']],
    "Training Set Precision": [train_results['precision']],
    "Training Set Recall": [train_results['recall']],
    "Training Set F1 Score": [train_results['f1_score']],
    "Training Set AUROC": [train_results['auc']],
    "Validation Set Accuracy": [val_results['accuracy']],
    "Validation Set Precision": [val_results['precision']],
    "Validation Set Recall": [val_results['recall']],
    "Validation Set F1 Score": [val_results['f1_score']],
    "Validation Set AUROC": [val_results['auc']],
    "Test Set Accuracy": [test_results['accuracy']],
    "Test Set Precision": [test_results['precision']],
    "Test Set Recall": [test_results['recall']],
    "Test Set F1 Score": [test_results['f1_score']],
    "Test Set AUROC": [test_results['auc']]
})

train_df = results_df[['Models', 'Time of Execution', 'Training Set Accuracy', 
                       'Training Set Precision', 'Training Set Recall', 'Training Set F1 Score', 'Training Set AUROC']]
val_df = results_df[['Models', 'Validation Set Accuracy', 'Validation Set Precision', 'Validation Set Recall', 
                     'Validation Set F1 Score', 'Validation Set AUROC']]
test_df = results_df[['Models', 'Test Set Accuracy', 'Test Set Precision', 'Test Set Recall', 
                      'Test Set F1 Score', 'Test Set AUROC']]

print("\nModel Evaluation Results:")
display_side_by_side([train_df, val_df, test_df], 
                     titles = ["Training Set", "Validation Set", "Test Set"], 
                     super_title = "Classification Models | Results")

In [ ]:
# Save to CSV
results_df.to_csv("ModelsEvaluation/BaselineModelEvaluation_1_29.03.2025.csv", index=False)                         ### Change the name of the file to save it

---

---

# **🔗 Bibliography/References**

**[[1]](https://)** AAAAAAAAAA